In [10]:
rm(list=ls())

library(rstan)

load("data_m.RData")

In [3]:
####################
# initial values
####################

# Initial values - 0
init0 <- list(psi = 7/77, lambda = 0.74)

# Initial values - 1
init1 <- list(psi = 0.9, lambda = 0.7)

# Initial values - 2
init2 <- list(psi = 0.6, lambda = 0.1)


In [4]:
fit1 <- stan(file = 'chlamydia_one_exponential_men.stan', data = chlamydia_dat_m, 
            iter = 22000, warmup = 2000, init=list(init0, init1, init2), chains = 3, seed=12345)


In file included from fileaff47c0a65c.cpp:8:
In file included from /Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/src/stan/model/model_header.hpp:4:
In file included from /Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/stan/math.hpp:4:
In file included from /Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/stan/math/rev/mat.hpp:4:
In file included from /Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/stan/math/rev/core.hpp:42:
/Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/stan/math/rev/core/set_zero_all_adjoints.hpp:14:17: warning: unused function 'set_zero_all_adjoints' [-Wunused-function]
    static void set_zero_all_adjoints() {
                ^
In file included from fileaff47c0a65c.cpp:8:
In file included from /Library/Frameworks/R.framework/Versions/3.2/Resources/library/StanHeaders/include/src/stan/model/mo

In [5]:
op_m <- extract(fit1)

In [6]:
# expectation of the deviance
Dbar_m <- mean(op_m$sumdev)

# Effective number of parameters (Spiegelhalter)
thetabar <- apply(op_m$theta,2,mean)
D_thetabar <- -2*sum(
    dbinom(chlamydia_dat_m$r, chlamydia_dat_m$n, thetabar, log=TRUE) -
    dbinom(chlamydia_dat_m$r, chlamydia_dat_m$n, chlamydia_dat_m$r/chlamydia_dat_m$n, log=TRUE)	
    )
p_DS_m <- Dbar_m - D_thetabar

# Effective number of parameters (Gelman)
p_DG_m <- 0.5 * var(op_m$sumdev)

# choose your p_D
p_Dm <- p_DS_m

# DIC
DIC_m <- p_Dm + Dbar_m

paste('Posterior mean residual deviance: ', round(Dbar_m,1), '.', collapse="")
paste('DIC: ', round(DIC_m,1), '.', collapse="")

[1] "Posterior mean residual deviance:  56.8 ."

[1] "DIC:  59 ."

## Three-compartment model

In [7]:
# Initial values - 0
init0 <- list(psi = 7/77, lambda_mid = 9.5, lambda_slow = 0.64, p1 = 0.19, p2=0.31)

# Initial values - 1
init1 <- list(psi = 0.9, lambda_mid = 15, lambda_slow = 0.1, p1 = 0.3, p2=0.31)

# Initial values - 2
init2 <- list(psi = 0.6,lambda_mid = 5, lambda_slow = 1, p1 = 0.6, p2=0.1)


In [ ]:
fit3 <- stan(file = 'chlamydia_three_exponentials_men.stan', data = chlamydia_dat_m, 
            iter = 22000, warmup = 2000, init=list(init0, init1, init2), chains = 3, seed=12345, verbose=FALSE)


## A model allowing for a distribution of clearance rates in the population

An alternative to the mixture-of-exponentials model is one in which the clearance rates take some distribution within the population: for example, a gamma distribution.

If $\lambda \sim$ Gamma( scale = k, shape = $\theta$ )

and X $\sim$ Exponential( rate = $\lambda$ )

then X $\sim$ Lomax( scale = 1/k, shape = $\theta$).

The proportion of incident infections which has cleared by time $t$ is therefore given by the cumulative distribution function (CDF) of the appropriately-parameterised Lomax distribution.

If the clearance rates of incident infections are gamma-distributed with scale $k$ and shape $\theta$, it can be shown that the clearance rates of prevalent infections are also gamma-distributed, with scale $k-1$ and shape $\theta$. The proportion of prevalent infections which has cleared by time $t$ is therefore given by the CDF of a Lomax distribution with scale $1/(k-1)$ and shape $\theta$.

In [15]:
# Fit the Lomax model

####################
# initial values
####################

# Initial values - 0
init0_lomax <- list(psi = 7/77, k = 1.01, theta = 0.5)

# Initial values - 1
init1_lomax <- list(psi = 0.9, k = 5, theta = 0.2)

# Initial values - 2
init2_lomax <- list(psi = 0.6, k = 10, theta = 0.1)

####################
# run the model
####################

fit_lomax <- stan(file = 'chlamydia_lomax_distribution_men.stan', data = chlamydia_dat_m, 
            iter = 22000, warmup = 2000, init=list(init0_lomax, init1_lomax, init2_lomax), chains = 3, seed=12345, 
            verbose = FALSE)

op_lomax <- extract(fit_lomax)



SAMPLING FOR MODEL 'chlamydia_lomax_distribution_men' NOW (CHAIN 1).

Chain 1, Iteration:     1 / 22000 [  0%]  (Warmup)
Chain 1, Iteration:  2001 / 22000 [  9%]  (Sampling)
Chain 1, Iteration:  4200 / 22000 [ 19%]  (Sampling)
Chain 1, Iteration:  6400 / 22000 [ 29%]  (Sampling)
Chain 1, Iteration:  8600 / 22000 [ 39%]  (Sampling)
Chain 1, Iteration: 10800 / 22000 [ 49%]  (Sampling)
Chain 1, Iteration: 13000 / 22000 [ 59%]  (Sampling)
Chain 1, Iteration: 15200 / 22000 [ 69%]  (Sampling)
Chain 1, Iteration: 17400 / 22000 [ 79%]  (Sampling)
Chain 1, Iteration: 19600 / 22000 [ 89%]  (Sampling)
Chain 1, Iteration: 21800 / 22000 [ 99%]  (Sampling)
Chain 1, Iteration: 22000 / 22000 [100%]  (Sampling)
 Elapsed Time: 0.106991 seconds (Warm-up)
               1.07011 seconds (Sampling)
               1.1771 seconds (Total)



The following numerical problems occured the indicated number of times after warmup on chain 1
                                                                                            count
Exception thrown at line 47: pareto_type_2_cdf: Scale parameter is inf, but must be finite!     9
Exception thrown at line 36: pareto_type_2_cdf: Shape parameter is inf, but must be finite!     1
When a numerical problem occurs, the Hamiltonian proposal gets rejected.
See http://mc-stan.org/misc/warnings.html#exception-hamiltonian-proposal-rejected
If the number in the 'count' column is small, do not ask about this message on stan-users.



SAMPLING FOR MODEL 'chlamydia_lomax_distribution_men' NOW (CHAIN 2).

Chain 2, Iteration:     1 / 22000 [  0%]  (Warmup)
Chain 2, Iteration:  2001 / 22000 [  9%]  (Sampling)
Chain 2, Iteration:  4200 / 22000 [ 19%]  (Sampling)
Chain 2, Iteration:  6400 / 22000 [ 29%]  (Sampling)
Chain 2, Iteration:  8600 / 22000 [ 39%]  (Sampling)
Chain 2, Iteration: 10800 / 22000 [ 49%]  (Sampling)
Chain 2, Iteration: 13000 / 22000 [ 59%]  (Sampling)
Chain 2, Iteration: 15200 / 22000 [ 69%]  (Sampling)
Chain 2, Iteration: 17400 / 22000 [ 79%]  (Sampling)
Chain 2, Iteration: 19600 / 22000 [ 89%]  (Sampling)
Chain 2, Iteration: 21800 / 22000 [ 99%]  (Sampling)
Chain 2, Iteration: 22000 / 22000 [100%]  (Sampling)
 Elapsed Time: 0.131467 seconds (Warm-up)
               1.21675 seconds (Sampling)
               1.34822 seconds (Total)



The following numerical problems occured the indicated number of times after warmup on chain 2
                                                                                            count
Exception thrown at line 47: pareto_type_2_cdf: Scale parameter is inf, but must be finite!     5
Exception thrown at line 36: pareto_type_2_cdf: Scale parameter is 0, but must be > 0!          1
When a numerical problem occurs, the Hamiltonian proposal gets rejected.
See http://mc-stan.org/misc/warnings.html#exception-hamiltonian-proposal-rejected
If the number in the 'count' column is small, do not ask about this message on stan-users.



SAMPLING FOR MODEL 'chlamydia_lomax_distribution_men' NOW (CHAIN 3).

Chain 3, Iteration:     1 / 22000 [  0%]  (Warmup)
Chain 3, Iteration:  2001 / 22000 [  9%]  (Sampling)
Chain 3, Iteration:  4200 / 22000 [ 19%]  (Sampling)
Chain 3, Iteration:  6400 / 22000 [ 29%]  (Sampling)
Chain 3, Iteration:  8600 / 22000 [ 39%]  (Sampling)
Chain 3, Iteration: 10800 / 22000 [ 49%]  (Sampling)
Chain 3, Iteration: 13000 / 22000 [ 59%]  (Sampling)
Chain 3, Iteration: 15200 / 22000 [ 69%]  (Sampling)
Chain 3, Iteration: 17400 / 22000 [ 79%]  (Sampling)
Chain 3, Iteration: 19600 / 22000 [ 89%]  (Sampling)
Chain 3, Iteration: 21800 / 22000 [ 99%]  (Sampling)
Chain 3, Iteration: 22000 / 22000 [100%]  (Sampling)
 Elapsed Time: 0.112356 seconds (Warm-up)
               1.13415 seconds (Sampling)
               1.2465 seconds (Total)



The following numerical problems occured the indicated number of times after warmup on chain 3
                                                                                            count
Exception thrown at line 47: pareto_type_2_cdf: Scale parameter is inf, but must be finite!    12
Exception thrown at line 36: pareto_type_2_cdf: Shape parameter is 0, but must be > 0!          1
When a numerical problem occurs, the Hamiltonian proposal gets rejected.
See http://mc-stan.org/misc/warnings.html#exception-hamiltonian-proposal-rejected
If the number in the 'count' column is small, do not ask about this message on stan-users.
Warning message:
"There were 9 divergent transitions after warmup. Increasing adapt_delta above 0.8 may help. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup"Warning message:
"Examine the pairs() plot to diagnose sampling problems
"

Calculate the DIC, in order to compare the Lomax model to the mixture of exponentials:

In [17]:
####################
# DIC
####################

# expectation of the deviance
Dbar <- mean(op_lomax$sumdev)

# Effective number of parameters (Spiegelhalter)
thetabar <- apply(op_lomax$theta,2,mean)
D_thetabar <- -2*sum(
    dbinom(chlamydia_dat_m$r, chlamydia_dat_m$n, thetabar, log=TRUE) -
    dbinom(chlamydia_dat_m$r, chlamydia_dat_m$n, chlamydia_dat_m$r/chlamydia_dat_m$n, log=TRUE)
    )
p_DS <- Dbar - D_thetabar

# Effective number of parameters (Gelman)
p_DG <- 0.5 * var(op_lomax$sumdev)

# choose your p_D
p_D <- p_DS

# DIC
DIC <- p_D + Dbar
DIC


[1] 32.03101